In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp

In [2]:
spark = SparkSession.builder.appName("practice").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/09 20:08:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Dataset - https://data.cityofnewyork.us/Public-Safety/Bureau-of-Fire-Prevention-Inspections-Historical-/ssq6-fkht/about_data

In [4]:
dy = spark.read.csv(
    "/Users/siddp278/Desktop/projects/dummy data/spark-data/FDNY.csv",
    inferSchema=True, # needed, else all data is read as string
    header=True
)
dy.head(3)

[Row(ACCT_ID=1, ALPHA='P', ACCT_NUM=4812798, OWNER_NAME='BEN RIC FUR FASHIONS INC', LAST_VISIT_DT='03/23/2018', LAST_FULL_INSP_DT='03/23/2018', LAST_INSP_STAT='APPROVAL', PREM_ADDR='186-14 UNION TNPK', BIN='0', COMMUNITY BOARD=None, COUNCIL DISTRICT=None, BBL=None, LATITUDE=None, LONGITUDE=None, POSTCODE=None, BOROUGH=None, Number='186-14', Street='UNION TNPK', Census Tract=None, NTA=None),
 Row(ACCT_ID=2, ALPHA='P', ACCT_NUM=6486682, OWNER_NAME='RICHAL INC.', LAST_VISIT_DT='03/23/2018', LAST_FULL_INSP_DT='03/23/2018', LAST_INSP_STAT='APPROVAL', PREM_ADDR='179-10 UNION TNPK', BIN='0', COMMUNITY BOARD=None, COUNCIL DISTRICT=None, BBL=None, LATITUDE=None, LONGITUDE=None, POSTCODE=None, BOROUGH=None, Number='179-10', Street='UNION TNPK', Census Tract=None, NTA=None),
 Row(ACCT_ID=3, ALPHA='C', ACCT_NUM=6743934, OWNER_NAME='CROSSTOWN MGT. CORP.', LAST_VISIT_DT='03/29/2017', LAST_FULL_INSP_DT='03/29/2017', LAST_INSP_STAT='NOV(HOLD)', PREM_ADDR='179-18 UNION TNPK', BIN='0', COMMUNITY BOARD=N

In [5]:
dy.count()

400618

In [6]:
dy.columns

['ACCT_ID',
 'ALPHA',
 'ACCT_NUM',
 'OWNER_NAME',
 'LAST_VISIT_DT',
 'LAST_FULL_INSP_DT',
 'LAST_INSP_STAT',
 'PREM_ADDR',
 'BIN',
 'COMMUNITY BOARD',
 'COUNCIL DISTRICT',
 'BBL',
 'LATITUDE',
 'LONGITUDE',
 'POSTCODE',
 'BOROUGH',
 'Number',
 'Street',
 'Census Tract',
 'NTA']

In [7]:
dy.printSchema()

root
 |-- ACCT_ID: integer (nullable = true)
 |-- ALPHA: string (nullable = true)
 |-- ACCT_NUM: integer (nullable = true)
 |-- OWNER_NAME: string (nullable = true)
 |-- LAST_VISIT_DT: string (nullable = true)
 |-- LAST_FULL_INSP_DT: string (nullable = true)
 |-- LAST_INSP_STAT: string (nullable = true)
 |-- PREM_ADDR: string (nullable = true)
 |-- BIN: string (nullable = true)
 |-- COMMUNITY BOARD: integer (nullable = true)
 |-- COUNCIL DISTRICT: integer (nullable = true)
 |-- BBL: long (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- POSTCODE: double (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- Number: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- Census Tract: string (nullable = true)
 |-- NTA: string (nullable = true)



In [23]:
dy_subset = dy.select(['ACCT_ID', 'ALPHA', 'OWNER_NAME', 
                      'LAST_FULL_INSP_DT', 'BIN', 'LATITUDE','LONGITUDE',
                        'POSTCODE', 'Street', 'Census Tract', 'NTA'])

dy_subset.take(3)

[Row(ACCT_ID=1, ALPHA='P', OWNER_NAME='BEN RIC FUR FASHIONS INC', LAST_FULL_INSP_DT='03/23/2018', BIN='0', LATITUDE=None, LONGITUDE=None, POSTCODE=None, Street='UNION TNPK', Census Tract=None, NTA=None),
 Row(ACCT_ID=2, ALPHA='P', OWNER_NAME='RICHAL INC.', LAST_FULL_INSP_DT='03/23/2018', BIN='0', LATITUDE=None, LONGITUDE=None, POSTCODE=None, Street='UNION TNPK', Census Tract=None, NTA=None),
 Row(ACCT_ID=3, ALPHA='C', OWNER_NAME='CROSSTOWN MGT. CORP.', LAST_FULL_INSP_DT='03/29/2017', BIN='0', LATITUDE=None, LONGITUDE=None, POSTCODE=None, Street='UNION TNPK', Census Tract=None, NTA=None)]

In [24]:
# DATA CLEANING
# Removing null values
dy_subset = dy_subset.na.drop(how = "any", subset=['POSTCODE', 'Street', 
                              'ALPHA', 'LATITUDE', 'LONGITUDE'])
# providing placeholder values
dy_subset = dy_subset.na.fill("placeholder owner name", "OWNER_NAME")
dy_subset = dy_subset.na.fill("12/08/2024", 'LAST_FULL_INSP_DT')
dy_subset.count()

378721

In [25]:
# DATA PROCESSING
# Chaning the date stamp from string to date format.
dy_subset.select(to_timestamp(dy_subset['LAST_FULL_INSP_DT']
    , 'yyyy-MM-dd').alias('dt'))

DataFrame[dt: timestamp]

In [26]:
dy_subset.take(3)

[Row(ACCT_ID=395, ALPHA='N', OWNER_NAME='NYC ECONOM.DEVEL. CORP', LAST_FULL_INSP_DT='09/28/2018', BIN='1000003', LATITUDE=40.701047, LONGITUDE=-74.011672, POSTCODE=10004.0, Street='SOUTH ST', Census Tract='9', NTA='Battery Park City-Lower Manhattan'),
 Row(ACCT_ID=416, ALPHA='S', OWNER_NAME='BEDFORD CONSTRUCTION', LAST_FULL_INSP_DT='07/21/2014', BIN='1000003', LATITUDE=40.701047, LONGITUDE=-74.011672, POSTCODE=10004.0, Street='SOUTH ST', Census Tract='9', NTA='Battery Park City-Lower Manhattan'),
 Row(ACCT_ID=422, ALPHA='S', OWNER_NAME='GMC CONTRACTING SERV', LAST_FULL_INSP_DT='08/26/2014', BIN='1000003', LATITUDE=40.701047, LONGITUDE=-74.011672, POSTCODE=10004.0, Street='SOUTH ST', Census Tract='9', NTA='Battery Park City-Lower Manhattan')]

In [27]:
dy_subset.createOrReplaceTempView("fdny_table")

In [28]:
result = spark.sql(
        "select OWNER_NAME, ALPHA from fdny_table where ALPHA='N'"
)
result.show()

+--------------------+-----+
|          OWNER_NAME|ALPHA|
+--------------------+-----+
|NYC ECONOM.DEVEL....|    N|
|TRIZEC HAHN 1 NY ...|    N|
| ONE NY PLAZA CO LLC|    N|
|BROOKFIELD PROPER...|    N|
|BROOKFIELD PROPER...|    N|
|BROOKFIELD PROPER...|    N|
|     SR PROP MANAGER|    N|
|BROOKFIELD PROPER...|    N|
|BROOKFIELD PROPER...|    N|
|BROOKFIELD PROPER...|    N|
|BROOKFIELD PROPER...|    N|
|BROOKFIELD PROPER...|    N|
|BROOKFIELD PROPER...|    N|
|BROOKFIELD PROPER...|    N|
|BROOKFIELD PROPER...|    N|
|BROOKFIELD PROPER...|    N|
|BROOKFIELD PROPER...|    N|
|BROOKFIELD PROPER...|    N|
|BROOKFIELD PROPER...|    N|
|BROOKFIELD PROPER...|    N|
+--------------------+-----+
only showing top 20 rows

